In [5]:
import numpy as np
import scipy
import pandas
import torch
import utils
from utils import generate_state_space_Matern_23
from scipy import linalg
from utils import build_id_key_table


In [2]:
data_file = '../processed_data/beijing_15k.npy'
data_dict = np.load(data_file, allow_pickle=True).item()

fold=0

# here should add one more data-loader class



hyper_dict={}
hyper_dict['ls'] = 0.5
hyper_dict['var'] = 0.1
hyper_dict['device'] = torch.device("cpu")
hyper_dict['R_U'] = 5 # dim of each node embedding
hyper_dict['c'] = 1.0 # diffusion rate

F,P_inf = utils.generate_state_space_Matern_23(data_dict,hyper_dict)

In [3]:
# data process:

class Bayes_diffu_tensor():
    def __init__(self,data_dict,hyper_dict):


        # hyper-paras
        self.epoch = hyper_dict['epoch'] # passing epoch
        self.R_U = hyper_dict['R_U'] # rank of latent factor of embedding
        self.device = hyper_dict['device']
        
        self.a0 = hyper_dict['a0']
        self.b0 = hyper_dict['b0']
        
        

        # data-dependent paras
        self.data_dict = data_dict
        
        self.ind_tr = data_dict['ind_tr']
        self.y_tr = data_dict['y_tr'].to(self.device) # N*1
        self.N = len(data_dict['y_tr'])
        self.ndims = data_dict['ndims']
        self.num_nodes = sum(self.ndims)
        
        self.train_time_ind =data_dict['tr_T_disct'] # N*1
        self.test_time_ind = data_dict['te_T_disct'] # N*1
        
        self.time_uni = data_dict['time_uni'] # N_time*1
        self.N_time = data_dict['N_time']  
        
        self.time_id_table = data_dict['time_id_table']
        self.F = data_dict['F'].to(self.device) # transition matrix
        self.P_inf = data_dict['P_inf'].to(self.device) # stationary covar
        
    
            
        # init the message factor of llk term (U_llk, tau)
        # and transition term (U_f: U_forard, U_b: U_backward)
        
        # actually, it's the massage from llk-factor -> variabel U
        # for here, we arrange the U-msg in mode-node way for efficient indexing in LLK
        self.msg_U_llk_m = [[torch.rand(ndim,self.R_U).double().to(self.device) \
            for ndim in self.ndims] ] * self.N_time
        
        self.msg_U_llk_v = [[torch.ones(ndim,self.R_U).double().to(self.device) \
            for ndim in self.ndims] ] * self.N_time
        
        self.msg_tau_a = torch.ones(self.N_time,1).to(self.device)
        self.msg_tau_b = torch.ones(self.N_time,1).to(self.device)
        
        # actually, it's the massage from transition-factor -> variabel U
        # for here, we arrange the U-msg in concat-all way for efficient computing in transition
        self.msg_U_f_m = [torch.rand(self.num_nodes,self.R_U).double().to(self.device)] * self.N_time
        self.msg_U_f_v = [torch.ones(self.num_nodes,self.R_U).double().to(self.device)] * self.N_time
        
        self.msg_U_b_m = [torch.rand(self.num_nodes,self.R_U).double().to(self.device)] * self.N_time
        self.msg_U_b_v = [torch.ones(self.num_nodes,self.R_U).double().to(self.device)] * self.N_time 


    
        # init the calibrating factors / q_del in draft
        
        # actually, it's the massage from variabel U -> llk-factor
        self.msg_U_llk_m_del = [[torch.rand(ndim,self.R_U).double().to(self.device) \
            for ndim in self.ndims] ] * self.N_time
        
        self.msg_U_llk_v_del = [[torch.ones(ndim,self.R_U).double().to(self.device) \
            for ndim in self.ndims] ] * self.N_time
        
        # as the computing of msg_tau_del is trival, we don't assign extra varb gor it  
        
        # actually, it's the massage from variabel U -> trans-factor
        self.msg_U_f_m_del = [torch.rand(self.num_nodes,self.R_U).double().to(self.device)] * self.N_time
        self.msg_U_f_v_del = [torch.ones(self.num_nodes,self.R_U).double().to(self.device)] * self.N_time
        
        self.msg_U_b_m_del = [torch.rand(self.num_nodes,self.R_U).double().to(self.device)] * self.N_time
        self.msg_U_b_v_del = [torch.ones(self.num_nodes,self.R_U).double().to(self.device)] * self.N_time 
        
    
        # init the post. U
        
        self.post_U_m = [[torch.rand(ndim,self.R_U).double().to(self.device) \
            for ndim in self.ndims] ] * self.N_time
        
        self.post_U_v = [[torch.ones(ndim,self.R_U).double().to(self.device) \
            for ndim in self.ndims] ] * self.N_time
        
        # uid-data table 
        self.uid_table, self.data_table = build_id_key_table(self.nmod,self.ind_tr) 
        

def msg_update_U_llk(self,T):
    pass
    
def msg_update_U_trans(self,T,mode='forward'):
    pass

def msg_update_tau(self,T):
    pass

def msg_update_U_llk_del(self,T):
    pass
    
def msg_update_U_trans_del(self,T):
    pass

def post_update_U(self,T):
    pass
    
def post_update_tau(self):
    pass
    
        
        
        
        
        
        

In [3]:
F,P_inf = utils.generate_state_space_Matern_23(data_dict,hyper_dict,fold)



In [9]:
a = [torch.rand(3,3,3),torch.rand(2,3,3)]
torch.cat(a,0).shape

torch.Size([5, 3, 3])

In [11]:
torch.div(torch.rand(3,3,3),torch.rand(3,3,3))

tensor([[[ 0.2385,  0.8594, 14.5998],
         [ 0.8539,  0.3681,  1.2468],
         [ 3.4039,  0.8496,  2.9541]],

        [[ 1.2068,  0.9472,  1.6964],
         [ 7.6235,  1.2204,  0.2621],
         [ 0.3200,  1.7626,  1.8560]],

        [[ 1.6478,  1.5325,  0.6460],
         [ 0.4114,  4.9742,  0.3559],
         [ 3.2568,  0.8296,  1.7869]]])

In [10]:
for item in list(np.unique(data_dict['t_te'])):
    if item not in list(np.unique(data_dict['t'])):
        print('no')

In [9]:
0.0 in list(np.unique(data_dict['t']))

True